# SageMaker Endpoint에 사전 훈련된 모델을 호스팅 후 Object Detection 수행하기 (PyTorch)
---

Amazon SageMaker에서 추론(inference)을 수행하려면 반드시 SageMaker에서 먼저 훈련을 수행해야 하나요? 그렇지 않습니다.<br>
만약 여러분이 SageMaker에서 추론만 수행하고 싶다면, 여러분의 온프레미스(on-premise)에서 훈련한 모델이나 공개 모델 저장소(model zoo)에 저장되어 있는 사전 훈련된(pre-trained) 모델들을 도커(Docker) 이미지 빌드 없이 그대로 SageMaker Endpoint에 배포할 수 있습니다. 여러분이 수행할 작업은 오로지 추론용 엔트리포인트(entrypoint)만 작성하는 것입니다.

이 노트북에서는 PyTorch API를 사용하여 사전 훈련된 `faster_rcnn` 모델을 SageMaker 엔드포인트에 배포 후, Object Detection을 수행합니다. 

## Pre-requisites

- 기본 용법: [PyTorch](https://gluon-cv.mxnet.io/tutorials/index.html)
- AWS 서비스: [AWS S3](https://docs.aws.amazon.com/s3/index.html), [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

In [ ]:
%load_ext autoreload
%autoreload 2

SageMaker SDK를 최신 버전으로 업그레이드합니다. 본 노트북은 SDK 2.x 버전 이상에서 구동해야 합니다.

In [ ]:
import sys, sagemaker, boto3
!{sys.executable} -m pip install -qU "sagemaker>=2.11.0"
print(sagemaker.__version__)

<br>

# 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트를 저장합니다.<br>

이 스크립트는 SageMaker 상에서 MXNet에 최적화된 추론 서버인 MMS(Multi Model Server)나 PyTorch에 최적화된 추론 서버인 torchserve를 쉽고 편하게 배포할 수 있는 high-level 툴킷인 SageMaker inference toolkit의 인터페이스를 사용하고 있으며, 여러분께서는 인터페이스에 정의된 핸들러(handler) 함수들만 구현하시면 됩니다. MXNet 및 PyTorch용 엔트리포인트(entrypoint) 인터페이스는 아래 두 가지 옵션 중 하나를 선택하면 되며, 본 예제에서는 Option 2.의 사용 예시를 보여줍니다.


### Option 1.
- `model_fn(model_dir)`: 딥러닝 네트워크 아키텍처를 정의하고 S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `input_fn(request_body, content_type)`: 입력 데이터를 전처리합니다. (예: request_body로 전송된 bytearray 배열을 PIL.Image로 변환 수 cropping, resizing, normalization등의 전처리 수행). content_type은 입력 데이터 종류에 따라 다양하게 처리 가능합니다. (예: application/x-npy, application/json, application/csv 등)
- `predict_fn(input_object, model)`: input_fn을 통해 들어온 데이터에 대해 추론을 수행합니다. 
- `output_fn(prediction, accept_type)`: predict_fn에서 받은 추론 결과를 추가 변환을 거쳐 프론트 엔드로 전송합니다. 

### Option 2. 
- `model_fn(model_dir)`: 딥러닝 네트워크 아키텍처를 정의하고 S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `transform_fn(model, request_body, content_type, accept_type)`: input_fn(), predict_fn(), output_fn()을 transform_fn()으로 통합할 수 있습니다.

In [ ]:
%%writefile code/inference_pytorch.py

# Built-Ins
import io, os, sys
import json
import subprocess, time

import numpy as np
from base64 import b64decode
from PIL import Image

import torch
import torchvision
from torchvision import datasets, transforms, models
from torchvision.models.detection import FasterRCNN
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    
def model_fn(model_dir=None):
    '''
    Loads the model into memory from storage and return the model.
    '''
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # load the model onto the computation device
    model = model.eval().to(device)    
    return model


def transform_fn(model, request_body, content_type='application/x-image', accept_type=None):
    '''
    Deserialize the request body and predicts on the deserialized object with the model from model_fn()
    '''
    if content_type == 'application/x-image':             
        img = np.array(Image.open(io.BytesIO(request_body)))
    elif content_type == 'application/x-npy':    
        img = np.frombuffer(request_body, dtype='uint8').reshape(137, 236)   
    else:
        raise ValueError(
            'Requested unsupported ContentType in content_type : ' + content_type)

    t0 = time.time()
    
    test_transforms = transforms.Compose([
        transforms.ToTensor()
    ])
    img_tensor = test_transforms(img).to(device)
    img_tensor = img_tensor.unsqueeze(0)
    
    with torch.no_grad():    
        result = model(img_tensor)

    t1 = time.time() - t0
    print("--- Elapsed time: %s secs ---" % t1)
    
    scores = result[0]['scores'].detach().cpu().numpy()
    bboxes = result[0]['boxes'].detach().cpu().numpy()
    cids = result[0]['labels'].detach().cpu().numpy()     
    
    outputs = json.dumps({'score': scores.tolist(), 
                       'bbox': bboxes.tolist(),
                         'cid': cids.tolist()})
    
    return outputs


Object Detection에 필요한 유틸리티 함수들을 정의합니다.

In [ ]:
%%writefile code/utils.py

def get_label_map(label_file):
    label_map = {}
    labels = open(label_file, 'r')
    
    for line in labels:
        line = line.rstrip("\n")
        ids = line.split(',')
        label_map[int(ids[0])] = ids[2] 
        
    return label_map


def get_label_map_imagenet(label_file):
    label_map = {}
    with open(label_file, 'r') as f:
        for line in f:
            key, val = line.strip().split(':')
            label_map[key] = val.replace(',', '')
    return label_map


def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    model_name = response['ProductionVariants'][0]['ModelName']

    client.delete_model(ModelName=model_name)    
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {endpoint_name}')    
    
    
def plot_bbox(img_resized, bboxes, scores, cids, class_info, framework='pytorch', threshold=0.5):

    import numpy as np
    import random
    import matplotlib.patches as patches
    import matplotlib.pyplot as plt
    
    if framework=='mxnet':
        img_np = img_resized.asnumpy()
        scores = scores.asnumpy()
        bboxes = bboxes.asnumpy()
        cids = cids.asnumpy()
    else:
        img_np = img_resized
        scores = np.array(scores)
        bboxes = np.array(bboxes)
        cids = np.array(cids)    

    # Get only results that are above the threshold. Default threshold is 0.5. 
    scores = scores[scores > threshold]
    num_detections = len(scores)
    bboxes = bboxes[:num_detections, :]
    cids = cids[:num_detections].astype('int').squeeze()

    # Get bounding-box colors
    cmap = plt.get_cmap('tab20b')
    colors = [cmap(i) for i in np.linspace(0, 1, 20)]
    random.seed(42)
    random.shuffle(colors)
    
    plt.figure()
    fig, ax = plt.subplots(1, figsize=(10,10))
    ax.imshow(img_np)

    if cids is not None:
        # Get unique class labels 
        unique_labels = set(list(cids.astype('int').squeeze()))
        unique_labels = np.array(list(unique_labels))
        n_cls_preds = len(unique_labels)
        bbox_colors = colors[:n_cls_preds]

        for b, cls_pred, cls_conf in zip(bboxes, cids, scores):
            x1, y1, x2, y2 = b[0], b[1], b[2], b[3]
            predicted_class = class_info[int(cls_pred)]
            label = '{} {:.2f}'.format(predicted_class, cls_conf)
            
            # Get box height and width
            box_h = y2 - y1
            box_w = x2 - x1

            # Add a box with the color for this class
            color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=3, edgecolor=color, facecolor='none')
            ax.add_patch(bbox)

            plt.text(x1, y1, s=label, color='white', verticalalignment='top',
                    bbox={'color': color, 'pad': 0})


<br>

# 2. Local Endpoint Inference
---

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있습니다. 따라서, 로컬 모드를 사용하여 실제 운영 환경에 배포하기 위한 추론 인스턴스를 시작하기 전에 노트북 인스턴스의 로컬 환경에서 모델을 배포하는 것을 권장합니다. 이를 로컬 모드 엔드포인트(Local Mode Endpoint)라고 합니다.

먼저, 로컬 모드 엔드포인트의 컨테이너 배포 이전에 로컬 환경 상에서 직접 추론을 수행하여 결과를 확인하고, 곧바로 로컬 모드 엔드포인트를 배포해 보겠습니다.

In [ ]:
import os
import json
import numpy as np
from io import BytesIO
from PIL import Image
from src.inference_pytorch import model_fn, transform_fn
from src.utils import get_label_map, delete_endpoint, plot_bbox

label_map = get_label_map('files/coco_labels.txt')
label_list = list(label_map.values())
print(label_map)

path = "./images/test"
img_list = os.listdir(path)
img_path_list = [os.path.join(path, img) for img in img_list]

test_idx = 0
img_path = img_path_list[test_idx]

with open(img_path, mode='rb') as file:
    img_byte = bytearray(file.read())

## 2.1. Local Inference without Endpoint

로컬 모드 엔드포인트 배포 이전에 로컬 환경상에서 아래와 같이 추론을 수행하면서 디버깅을 수행할 수 있습니다.

In [15]:
model = model_fn('tfs-workshop/model_artifact')
response_body = transform_fn(model, img_byte)
outputs = json.loads(response_body)

2022-10-18 13:59:10 | INFO | fairseq.file_utils | Archive name 'tfs-workshop/model_artifact' was not found in archive name list. We assumed 'tfs-workshop/model_artifact' was a path or URL but couldn't find any file associated to this path or URL.
2022-10-18 13:59:10 | INFO | fairseq.file_utils | Archive name 'model_dir' was not found in archive name list. We assumed 'model_dir' was a path or URL but couldn't find any file associated to this path or URL.


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
img = Image.open(img_path)
plot_bbox(img, outputs['bbox'], outputs['score'], outputs['cid'], class_info=label_map)

## 2.2. Local Mode Endpoint

이제 로컬 모드로 배포를 수행합니다.

Fine-tuning 없이 곧바로 사전 훈련된 모델을 사용할 것이므로 `model.tar.gz`는 0바이트의 빈 파일로 구성합니다.
만약 온프레미스에서 fine-tuning을 수행한 모델을 사용하고 싶다면, 모델 파라메터(예: model.pth)들을 `model.tar.gz`로 압축하세요.

In [ ]:
f = open("model.pth", 'w')
f.close()
!tar -czf model.tar.gz model.pth

In [ ]:
import os
import time
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer
from sagemaker.pytorch.model import PyTorchModel
role = 'sage-maker-full-access'

In [ ]:
endpoint_name = "local-endpoint-pytorch-{}".format(int(time.time()))
local_model_path = f'file://{os.getcwd()}/model_artifact/model.tar.gz'


아래 코드 셀을 실행 후, 로그를 확인해 보세요. torchserve 대한 세팅값들을 확인하실 수 있습니다.

```bash
algo-1-9m80o_1  | ['torchserve', '--start', '--model-store', '/.sagemaker/ts/models', '--ts-config', '/etc/sagemaker-ts.properties', '--log-config', '/opt/conda/lib/python3.6/site-packages/sagemaker_pytorch_serving_container/etc/log4j.properties', '--models', 'model.mar']
algo-1-9m80o_1  | 2020-12-29 14:07:56,008 [INFO ] main org.pytorch.serve.ModelServer - 
algo-1-9m80o_1  | Torchserve version: 0.2.1
algo-1-9m80o_1  | TS Home: /opt/conda/lib/python3.6/site-packages
algo-1-9m80o_1  | Current directory: /
algo-1-9m80o_1  | Temp directory: /home/model-server/tmp
...
```

**[Note]** SageMaker SDK v2부터 serializer와 deserializer의 content_type에 대한 property를 직접 지정하는 것이 아니라 직접 클래스 인스턴스를 생성해야 합니다. content_type이 `'application/x-image'`일 경우는 `IdentitySerializer` 클래스를 사용하시면 됩니다.

In [ ]:
model = PyTorchModel(model_data=local_model_path,
                     role=role,
                     entry_point='inference_pytorch.py', 
                     source_dir='src',
                     framework_version='1.6.0',
                     py_version='py3',
                     env={'SAGEMAKER_MODEL_SERVER_WORKERS': '1'})

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local',
    serializer=JSONSerializer(content_type='application/json'),
    deserializer=JSONDeserializer()
)

ecosfogyr3-algo-1-c38hy  | 2022-10-18 04:17:41,053 [INFO ] pool-1-thread-10 ACCESS_LOG - /172.19.0.1:44280 "GET /ping HTTP/1.1" 200 0
ecosfogyr3-algo-1-c38hy  | 2022-10-18 04:17:41,054 [INFO ] pool-1-thread-10 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:7c44c5cd3247,timestamp:null
ecosfogyr3-algo-1-c38hy  | 2022-10-18 04:23:25,707 [INFO ] pool-1-thread-11 ACCESS_LOG - /172.19.0.1:44348 "GET /ping HTTP/1.1" 200 1
ecosfogyr3-algo-1-c38hy  | 2022-10-18 04:23:25,707 [INFO ] pool-1-thread-11 TS_METRICS - Requests2XX.Count:1|#Level:Host|#hostname:7c44c5cd3247,timestamp:null
ecosfogyr3-algo-1-c38hy  | 2022-10-18 04:27:42,101 [INFO ] epollEventLoopGroup-2-1 org.pytorch.serve.ModelServer - Inference model server stopped.
ecosfogyr3-algo-1-c38hy  | 2022-10-18 04:27:42,102 [INFO ] epollEventLoopGroup-2-2 org.pytorch.serve.ModelServer - Metrics model server stopped.
ecosfogyr3-algo-1-c38hy  | 2022-10-18 04:27:44,118 [INFO ] main org.pytorch.serve.ModelServer - Torchserve stopped.
ecosfo

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/yms9654/workspace/fairseq2/.venv/lib/python3.10/site-packages/sagemaker/local/image.py", line 854, in run
    _stream_output(self.process)
  File "/home/yms9654/workspace/fairseq2/.venv/lib/python3.10/site-packages/sagemaker/local/image.py", line 916, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/yms9654/workspace/fairseq2/.venv/lib/python3.10/site-packages/sagemaker/local/image.py", line 859, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp3f6j3jnd/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 137


ecosfogyr3-algo-1-c38hy exited with code 137
Aborting on container exit...


로컬에서 컨테이너를 배포했기 때문에 컨테이너가 현재 실행 중임을 확인할 수 있습니다.

In [ ]:
!docker ps
# print(model.endpoint_name)

### SageMaker SDK로 엔드포인트 호출

In [ ]:
outputs = predictor.predict(img_byte)

In [ ]:
plot_bbox(img, outputs['bbox'], outputs['score'], outputs['cid'], class_info=label_map)

### Boto3 API로 엔드포인트 호출

위의 코드 셀처럼 SageMaker SDK `predict()` 메서드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.<br>
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리
SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

참고로 `invoke_endpoint()` 호출을 위한 런타임 클라이언트 인스턴스 생성 시, 로컬 배포 모드에서는 `sagemaker.local.LocalSagemakerRuntimeClient()`를 호출해야 합니다.

In [ ]:
import sagemaker
import boto3

runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = model.endpoint_name

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/x-image',
    Accept='application/json',
    Body=img_byte
    )
outputs = json.loads(response['Body'].read().decode())

In [ ]:
plot_bbox(img, outputs['bbox'], outputs['score'], outputs['cid'], class_info=label_map)

### Local Mode Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.<br>
참고로, 노트북 인스턴스에서 추론 컨테이너를 배포했기 때문에 엔드포인트를 띄워 놓아도 별도로 추가 요금이 과금되지는 않습니다. 


```python
# SageMaker SDK
predictor.delete_endpoint()

# Boto3 API
client.delete_model(ModelName=model_name)    
client.delete_endpoint(EndpointName=endpoint_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_name)  

# 직접 삭제
!docker rm $(docker ps -a -q)
```

In [ ]:
predictor.delete_endpoint()

<br>

# 3. SageMaker Hosted Endpoint Inference
---

이제 실제 운영 환경에 엔드포인트 배포를 수행해 보겠습니다. 로컬 모드 엔드포인트와 대부분의 코드가 동일하며, 모델 아티팩트 경로(`model_data`)와 인스턴스 유형(`instance_type`)만 변경해 주시면 됩니다.

#### [주의] 아래 코드 셀을 그대로 실행하지 마시고 bucket 이름을 반드시 수정해 주세요.
```python
bucket = '[YOUR-S3-BUCKET]' # as-is
bucket = 'sagemaker-hol-daekeun' # to-be
```

In [16]:
role = 'sage-maker-full-access'
bucket = 'npx-translation-models' # bucket 이름을 반드시 수정해 주세요.
# bucket = sagemaker.Session().default_bucket() # SageMaker에서 자동으로 생성되는 bucket

In [17]:
%%bash -s "$role" "$bucket"
ROLE=$1
BUCKET=$2

aws s3 cp model_artifact/model.tar.gz s3://$BUCKET/model.tar.gz

2022-10-18 15:53:07 | ERROR | asyncio | Task exception was never retrieved
future: <Task finished name='Task-7' coro=<ScriptMagics.shebang.<locals>._handle_stream() done, defined at /home/yms9654/workspace/fairseq2/.venv/lib/python3.10/site-packages/IPython/core/magics/script.py:211> exception=ValueError('Separator is not found, and chunk exceed the limit')>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/streams.py", line 525, in readline
    line = await self.readuntil(sep)
  File "/usr/lib/python3.10/asyncio/streams.py", line 603, in readuntil
    raise exceptions.LimitOverrunError(
asyncio.exceptions.LimitOverrunError: Separator is not found, and chunk exceed the limit

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yms9654/workspace/fairseq2/.venv/lib/python3.10/site-packages/IPython/core/magics/script.py", line 213, in _handle_stream
    line = (await stream.readline()).decode("utf8")
  

SageMaker가 관리하는 배포 클러스터를 프로비저닝하고 추론 컨테이너를 배포하기 때문에, 추론 서비스를 시작하는 데에는 약 5~10분 정도 소요됩니다.

In [ ]:
%%time

model_path = 's3://{}/model.tar.gz'.format(bucket)
endpoint_name = "endpoint-pytorch-object-detection-{}".format(int(time.time()))

model = PyTorchModel(model_data=model_path,
                     role=role,
                     entry_point='inference_pytorch.py', 
                     source_dir='src',
                     framework_version='1.6.0',
                     py_version='py3',
                     env={'SAGEMAKER_MODEL_SERVER_WORKERS': '1'})

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.c5.large',
    serializer=IdentitySerializer(content_type='application/x-image'),
    deserializer=JSONDeserializer()
)

In [ ]:
outputs = predictor.predict(img_byte)

In [ ]:
plot_bbox(img, outputs['bbox'], outputs['score'], outputs['cid'], class_info=label_map)

### Endpoint Clean-up

SageMaker Endpoint로 인한 과금을 막기 위해, 본 핸즈온이 끝나면 반드시 Endpoint를 삭제해 주시기 바랍니다.

In [ ]:
predictor.delete_endpoint()